In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [8]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=3,
    lora_dropout=0.05,
    bias='lora_only',
    target_modules=['query', 'key', 'value', 'attention.output.dense'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 704,260 || all params: 125,316,104 || trainable%: 0.5620


In [9]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): Mo

In [10]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=3000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.359700,1.303991,0.782813
200,0.953300,0.419303,0.873437
300,0.393900,0.345329,0.889062
400,0.297000,0.342828,0.892188
500,0.321000,0.332115,0.906250
600,0.309100,0.325713,0.900000
700,0.307700,0.330074,0.903125
800,0.333300,0.318382,0.904687
900,0.308500,0.313602,0.907813
1000,0.296100,0.310566,0.906250


In [15]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2700'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [01:00<00:00,  4.13it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2700.csv


In [13]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-3000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3597,1.303991,0.400000,0.782813,0.000048,0.013405,0.055709,1.042722,-0.382812,0.510978
1,200,0.9533,0.419303,0.788393,0.873437,0.000047,0.026810,0.533997,2.273534,-0.085045,0.902632
2,300,0.3939,0.345329,0.874554,0.889062,0.000045,0.040214,0.048571,1.140652,-0.014509,0.983681
3,400,0.2970,0.342828,0.900893,0.892188,0.000043,0.053619,-0.045828,0.866323,0.008705,1.009757
4,500,0.3210,0.332115,0.893304,0.906250,0.000042,0.067024,-0.011115,0.966532,-0.012946,0.985714
5,600,0.3091,0.325713,0.897321,0.900000,0.000040,0.080429,-0.016613,0.948994,-0.002679,0.997024
6,700,0.3077,0.330074,0.901786,0.903125,0.000038,0.093834,-0.022374,0.932216,-0.001339,0.998517
7,800,0.3333,0.318382,0.885714,0.904687,0.000037,0.107239,0.014918,1.046855,-0.018973,0.979028
8,900,0.3085,0.313602,0.899554,0.907813,0.000035,0.120643,-0.005102,0.983729,-0.008259,0.990902
9,1000,0.2961,0.310566,0.901786,0.906250,0.000033,0.134048,-0.014466,0.953421,-0.004464,0.995074
